If you want, I’ll now give:

* 🔥 **Notebook-2 code**
* 📊 **Final comparison table (scratch vs pretrained)**
* 🧠 **Viva explanation**

Just say **“give pretrained notebook”**


In [1]:
!pip install transformers rouge-score sentencepiece torch --quiet


In [2]:
import pandas as pd
import numpy as np
import torch

from transformers import pipeline
from rouge_score import rouge_scorer


In [3]:
df = pd.read_csv("C:/Users/Malathi M/OneDrive/Documents/MDTE25/guvi final project/Main project/news.tsv.zip", sep="\t")

In [9]:
df = df.dropna(subset=["Headline", "News body", "Category"])

df["article"] = df["Headline"].astype(str) + " " + df["News body"].astype(str)
df["summary"] = df["Headline"].astype(str)

# Use SAME 50% sampling as notebook 1
df = df.sample(frac=0.5, random_state=42).reset_index(drop=True)

df.head()

,News ID,Category,Topic,Headline,News body,Title entity,Entity content,article,summary
0,N32957,foodanddrink,videos,How to Make Spaghetti Pudding,"Similar to bread pudding or rice pudding, this...",{},{},How to Make Spaghetti Pudding Similar to bread...,How to Make Spaghetti Pudding
1,N15327,sports,basketball_nba,Report: Dolan was not interested in paying max...,As if that's the only thing it hinged on NBA f...,"{'Dolan': 'Pat Dolan', 'Durant': 'Kevin Durant'}","{'Pat Dolan': {'type': 'item', 'id': 'Q7143387...",Report: Dolan was not interested in paying max...,Report: Dolan was not interested in paying max...
2,N117490,news,newspolitics,House panel votes to subpoena Kellyanne Conway,The House Oversight Committee has voted to sub...,{},{},House panel votes to subpoena Kellyanne Conway...,House panel votes to subpoena Kellyanne Conway
3,N100097,finance,markets,The 'Texas Miracle' Missed Most of Texas,"LONGVIEW, Tex. On the eastern plains of Texa...","{""'Texas Miracle'"": 'Miracle', 'Texas': 'Texas'}","{'Miracle': {'type': 'item', 'id': 'Q79945462'...",The 'Texas Miracle' Missed Most of Texas LONGV...,The 'Texas Miracle' Missed Most of Texas
4,N114474,autos,autosconvertibles,2020 BMW Z4 First Drive Review | More cerebral...,Much has changed with the compact two-seater c...,{'BMW': 'BMW'},"{'BMW': {'type': 'item', 'id': 'Q26678', 'labe...",2020 BMW Z4 First Drive Review | More cerebral...,2020 BMW Z4 First Drive Review | More cerebral...


In [5]:
baseline_df = pd.read_csv("C:/Users/Malathi M/OneDrive/Documents/MDTE25/guvi final project/Main project/summarization/from_scratch_summarization_results.csv")
baseline_df


,Model,ROUGE-1,ROUGE-2,ROUGE-L,Average
0,TF-IDF,0.159553,0.148539,0.159553,0.155882
1,TextRank,0.159553,0.148539,0.159553,0.155882
2,Seq2Seq,0.124501,0.057310,0.112941,0.098250


In [10]:
from transformers import (
    BertTokenizer, BertModel,
    T5Tokenizer, T5ForConditionalGeneration
)

In [11]:
bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
bert_model = BertModel.from_pretrained("bert-base-uncased")
bert_model.eval()


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

In [12]:
def bert_sentence_embedding(sentence):
    inputs = bert_tokenizer(
        sentence,
        return_tensors="pt",
        truncation=True,
        max_length=64
    )
    with torch.no_grad():
        outputs = bert_model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze().numpy()


In [14]:
from sklearn.metrics.pairwise import cosine_similarity
from rouge_score import rouge_scorer
from nltk.tokenize import sent_tokenize

In [15]:
def bert_extractive_summarize(text, top_n=3):
    sentences = sent_tokenize(text)

    if len(sentences) <= top_n:
        return " ".join(sentences)

    embeddings = np.array(
        [bert_sentence_embedding(s) for s in sentences]
    )

    doc_embedding = embeddings.mean(axis=0).reshape(1, -1)
    scores = cosine_similarity(embeddings, doc_embedding).flatten()

    ranked = scores.argsort()[::-1][:top_n]
    ranked = sorted(ranked)

    return " ".join([sentences[i] for i in ranked])


In [16]:
t5_tokenizer = T5Tokenizer.from_pretrained("t5-small")
t5_model = T5ForConditionalGeneration.from_pretrained("t5-small")
t5_model.eval()


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [17]:
def t5_summarize(text, max_len=120):
    input_text = "summarize: " + text[:512]

    inputs = t5_tokenizer(
        input_text,
        return_tensors="pt",
        truncation=True
    )

    with torch.no_grad():
        summary_ids = t5_model.generate(
            inputs["input_ids"],
            max_length=max_len,
            min_length=40,
            num_beams=2,
            early_stopping=True
        )

    return t5_tokenizer.decode(
        summary_ids[0],
        skip_special_tokens=True
    )


In [18]:
scorer = rouge_scorer.RougeScorer(
    ["rouge1", "rouge2", "rougeL"],
    use_stemmer=True
)

def evaluate_model(fn, df, samples=30):
    r1, r2, rL = [], [], []

    for i in range(samples):
        ref = df.iloc[i]["summary"]
        pred = fn(df.iloc[i]["article"])

        scores = scorer.score(ref, pred)
        r1.append(scores["rouge1"].fmeasure)
        r2.append(scores["rouge2"].fmeasure)
        rL.append(scores["rougeL"].fmeasure)

    return np.mean(r1), np.mean(r2), np.mean(rL)


In [19]:
bert_scores = evaluate_model(bert_extractive_summarize, df)
t5_scores = evaluate_model(t5_summarize, df)

results = pd.DataFrame([
    {
        "Model": "BERT-base (Extractive)",
        "ROUGE-1": bert_scores[0],
        "ROUGE-2": bert_scores[1],
        "ROUGE-L": bert_scores[2]
    },
    {
        "Model": "T5-small (Abstractive)",
        "ROUGE-1": t5_scores[0],
        "ROUGE-2": t5_scores[1],
        "ROUGE-L": t5_scores[2]
    }
])

results["Average"] = results[["ROUGE-1", "ROUGE-2", "ROUGE-L"]].mean(axis=1)
results.round(4)


,Model,ROUGE-1,ROUGE-2,ROUGE-L,Average
0,BERT-base (Extractive),0.1665,0.1189,0.1520,0.1458
1,T5-small (Abstractive),0.3012,0.2217,0.2782,0.2670


In [20]:
best_model = results.sort_values("Average", ascending=False).iloc[0]

best_config = {
    "model": best_model["Model"],
    "rouge_avg": best_model["Average"]
}

import joblib
joblib.dump(best_config, "best_transformer_summarizer.pkl")


['best_transformer_summarizer.pkl']

REFERENCE

In [6]:
MODELS = {
    "T5-small": "t5-small",
    "T5-base": "t5-base",
    "BART-base": "facebook/bart-base"
}


In [7]:
scorer = rouge_scorer.RougeScorer(
    ["rouge1", "rouge2", "rougeL"],
    use_stemmer=True
)

def evaluate_transformer(model_name, df, samples=30):
    summarizer = pipeline(
        "summarization",
        model=model_name,
        tokenizer=model_name,
        device=-1  # CPU
    )

    r1, r2, rL = [], [], []

    for i in range(samples):
        text = df.iloc[i]["article"][:1500]  # truncate for safety
        ref = df.iloc[i]["summary"]

        pred = summarizer(
            text,
            max_length=80,
            min_length=30,
            do_sample=False
        )[0]["summary_text"]

        scores = scorer.score(ref, pred)
        r1.append(scores["rouge1"].fmeasure)
        r2.append(scores["rouge2"].fmeasure)
        rL.append(scores["rougeL"].fmeasure)

    return {
        "rouge1": np.mean(r1),
        "rouge2": np.mean(r2),
        "rougeL": np.mean(rL)
    }


In [ ]:
transformer_results = []

for name, model_id in MODELS.items():
    print(f"Evaluating {name}...")
    scores = evaluate_transformer(model_id, df)

    transformer_results.append({
        "Model": name,
        "rouge1": scores["rouge1"],
        "rouge2": scores["rouge2"],
        "rougeL": scores["rougeL"]
    })

transformer_df = pd.DataFrame(transformer_results)
transformer_df["Average"] = transformer_df[
    ["rouge1", "rouge2", "rougeL"]
].mean(axis=1)

transformer_df.round(4)


In [ ]:
combined_df = pd.concat(
    [baseline_df, transformer_df],
    ignore_index=True
)

combined_df = combined_df.round(4)
combined_df


In [ ]:
combined_df.to_csv(
    "final_summarization_comparison.csv",
    index=False
)

print("Saved → final_summarization_comparison.csv")


In [ ]:
best_row = combined_df.sort_values(
    "Average",
    ascending=False
).iloc[0]

best_model = {
    "model_name": best_row["Model"],
    "rouge1": best_row["rouge1"],
    "rouge2": best_row["rouge2"],
    "rougeL": best_row["rougeL"],
    "average": best_row["Average"],
    "model_type": "Transformer"
}

import joblib
joblib.dump(best_model, "best_transformer_summarizer.pkl")

best_model
